# Installing Libraries
이번에는 scikit-learn 말고도 Gradient Boosting 기반의 다른 알고리즘들을 사용해보자.

In [ ]:
!pip install -q scikit-learn==1.4.1.post1 numpy pandas altair xgboost lightgbm catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 7.1 MB/s eta 0:00:00


* [xgboost](https://xgboost.ai/): XGBoost
* [lightgbm](https://lightgbm.readthedocs.io/en/stable/): LightGBM
* [catboost](https://catboost.ai/): CatBoost

# Data Loading

지난 시간과 마찬가지로, **breast_cancer** 데이터셋을 활용해보자.

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import StratifiedShuffleSplit


data = load_breast_cancer()
X, y = data.data, data.target
I_train, I_test = next(StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=42).split(X, y))
X_train, y_train = X[I_train], y[I_train]
X_test, y_test = X[I_test], y[I_test]

# Random Forest
일단, Baseline으로 Bagging + Random Subspace인 Random Forest를 훈련시킬것이다.

In [ ]:
from sklearn.ensemble import RandomForestClassifier


model_rf = RandomForestClassifier(
    n_estimators=100, # 100개의 Decision Tree를 만든다
    max_depth=5, # 각 Decision Tree의 깊이는 5로 제한한다
    max_features='sqrt', # 30개의 특성값 공간 중 약 5개만 사용한다
    max_samples=0.5, # 전체 훈련 샘플 중 50%만 Bootstrap한다
    random_state=42
).fit(
    X_train, y_train
)

In [ ]:
from sklearn.metrics import accuracy_score


print('# Training performance')
print('- Random Forest: ', accuracy_score(y_train, model_rf.predict(X_train)))

print('# Validation performance')
print('- Random Forest: ', accuracy_score(y_test, model_rf.predict(X_test)))

# Training performance
- Random Forest:  0.9882697947214076
# Validation performance
- Random Forest:  0.9473684210526315


# AdaBoost

먼저 AdaBoost부터 시험해보자. 구현체는 [sklearn.ensemble.AdaBoostClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html)에 있다.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier


model_ab = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1, random_state=42), # 깊이 1의 Decision Tree, 즉 Stump를 활용한다. 기본값이 이렇게 설정되어 있다.
    n_estimators=100, # 최대 100개의 Stump를 만들 것이다. 하지만, 중간에 Loss가 0가 된다면, 더 이상 Stump를 생성하지 않고 훈련이 종료된다.
    learning_rate=1.0, # 각 Stump가 미치는 영향을 나타낸다. 일단은 기본값으로 설정하자.
    random_state=42
).fit(
    X_train, y_train
)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score


print('# Training performance')
print('- AdaBoost: ', accuracy_score(y_train, model_ab.predict(X_train)))

print('# Validation performance')
print('- AdaBoost: ', accuracy_score(y_test, model_ab.predict(X_test)))

# Training performance
- AdaBoost:  1.0
# Validation performance
- AdaBoost:  0.9605263157894737


다행스럽게도(?) Random Forest보다 훨씬 성능이 좋은 것을 알 수 있다.

# Gradient Boosting
이번엔, Gradient Boosting을 활용해보자. 기본적인 Gradient Boosting 알고리즘은 구현체는 [sklearn.ensemble.GradientBoostingClassifer](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier)에 구현되어 있다.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier


model_gb = GradientBoostingClassifier(
    n_estimators=100, # 최대 100개의 Decision Tree를 만들 것이다. 하지만, 중간에 Loss가 0가 된다면, 더 이상 Decision Tree를 생성하지 않고 훈련이 종료된다.
    learning_rate=0.1, # 각 Decision Tree가 결과에 미치는 영향을 나타낸다(Shrinkage). 과대적합을 막기 위해서 낮은 값으로 설정해보자
    subsample=0.8, # 각 Decision Tree 훈련에 사용할 샘플의 개수를 나타낸다. 80%만 활용해보자.
    max_depth=3, # Decision Tree의 깊이를 나타낸다. 많이 깊어지지 않도록 해보자.
    min_samples_leaf=1, # Decision Tree의 잎 노드에 필요한 샘플의 개수를 나타낸다. 깊이를 조절했으므로, 이건 그냥 1로 놔두자.
    max_features=1.0, # Random Subspace의 적용을 나타낸다. 전체 Feature를 다 사용하자.
    random_state=42
).fit(
    X_train, y_train
)

In [ ]:
from sklearn.metrics import accuracy_score


print('# Training performance')
print('- Gradient Boosting: ', accuracy_score(y_train, model_gb.predict(X_train)))

print('# Validation performance')
print('- Gradient Boosting: ', accuracy_score(y_test, model_gb.predict(X_test)))

# Training performance
- Gradient Boosting:  1.0
# Validation performance
- Gradient Boosting:  0.9605263157894737


AdaBoost와 같은 성능이 나왔다.

# XGBoost
이번엔 Gradient Boosting 기반의 응용 알고리즘들을 사용해보자. 먼저, XGBoost다.

초기 XGBoost는 자신들만의 데이터 구조(예. [xgboost.DMatrix](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.DMatrix)) 및 훈련 함수(예. [xgboost.train](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.train)) 등이 있었으나, 이제는 scikit-learn과 동일한 문법을 통해 XGBoost 모델을 훈련시킬 수 있도록 지원한다. 해당 구현체는 [xgboost.XGBClassifier](https://xgboost.readthedocs.io/en/stable/python/python_api.html#xgboost.XGBClassifier)에서 확인할 수 있다.

In [ ]:
from xgboost import XGBClassifier


model_xgb = XGBClassifier(
    n_estimators=100, # 최대 100개의 Decision Tree를 만들 것이다. 하지만, 중간에 Loss가 0가 된다면, 더 이상 Decision Tree를 생성하지 않고 훈련이 종료된다.
    learning_rate=0.1, # 각 Decision Tree가 결과에 미치는 영향을 나타낸다(Shrinkage). 과대적합을 막기 위해서 낮은 값으로 설정해보자
    subsample=0.8, # 각 Decision Tree 훈련에 사용할 샘플의 개수를 나타낸다. 80%만 활용해보자.
    max_depth=3, # Decision Tree의 깊이를 나타낸다. 많이 깊어지지 않도록 해보자.
    random_state=42
).fit(
    X_train, y_train
)

In [ ]:
from sklearn.metrics import accuracy_score


print('# Training performance')
print('- Gradient Boosting: ', accuracy_score(y_train, model_xgb.predict(X_train)))

print('# Validation performance')
print('- Gradient Boosting: ', accuracy_score(y_test, model_xgb.predict(X_test)))

# Training performance
- Gradient Boosting:  1.0
# Validation performance
- Gradient Boosting:  0.9692982456140351


지금까지 확인한 것 중 가장 좋은 성능이 나왔다.

위에서 설정한 초매개변수 이외에도, XGBoost에는 정말 많은 초매개변수가 존재한다. 보다 자세한 건 [공식 문서](https://xgboost.readthedocs.io/en/stable/parameter.html)를 확인하자.

# LightGBM
이번엔 LightGBM이다. 역시 LightGBM도 XGBoost와 유사하게 자신만의 인터페이스가 있었으나, scikit-learn과 동일한 인터페이스 또한 지원한다. LightGBM 기반 분류기는 [lightgbm.LGBMClassifier](https://lightgbm.readthedocs.io/en/stable/pythonapi/lightgbm.LGBMClassifier.html#)에 구현되어 있다.

In [ ]:
from lightgbm import LGBMClassifier


model_lgbm = LGBMClassifier(
    n_estimators=100, # 최대 100개의 Decision Tree를 만들 것이다. 하지만, 중간에 Loss가 0가 된다면, 더 이상 Decision Tree를 생성하지 않고 훈련이 종료된다.
    learning_rate=0.1, # 각 Decision Tree가 결과에 미치는 영향을 나타낸다(Shrinkage). 과대적합을 막기 위해서 낮은 값으로 설정해보자
    subsample=0.8, # 각 Decision Tree 훈련에 사용할 샘플의 개수를 나타낸다. 80%만 활용해보자.
    max_depth=3, # Decision Tree의 깊이를 나타낸다. 많이 깊어지지 않도록 해보자.
    random_state=42,
    verbosity=-1
).fit(
    X_train, y_train
)

In [ ]:
from sklearn.metrics import accuracy_score


print('# Training performance')
print('- Gradient Boosting: ', accuracy_score(y_train, model_lgbm.predict(X_train)))

print('# Validation performance')
print('- Gradient Boosting: ', accuracy_score(y_test, model_lgbm.predict(X_test)))

# Training performance
- Gradient Boosting:  1.0
# Validation performance
- Gradient Boosting:  0.9605263157894737


역시, 위에서 설정한 초매개변수 이외에도, LightGBM에는 정말 많은 초매개변수가 존재한다. 보다 자세한 건 [공식 문서](https://lightgbm.readthedocs.io/en/latest/Parameters.html)를 확인하자

# CatBoost
마지막으론 CatBoost이다. CatBoost 또한 scikit-learn과 동일한 인터페이스를 활용해서 모델을 훈련할 수 있다. 구현체는 [catboost.CatBoostClassifier](https://catboost.ai/en/docs/concepts/python-reference_catboostclassifier)다.

In [ ]:
from catboost import CatBoostClassifier


model_cat = CatBoostClassifier(
    n_estimators=100, # 최대 100개의 Decision Tree를 만들 것이다. 하지만, 중간에 Loss가 0가 된다면, 더 이상 Decision Tree를 생성하지 않고 훈련이 종료된다.
    learning_rate=0.1, # 각 Decision Tree가 결과에 미치는 영향을 나타낸다(Shrinkage). 과대적합을 막기 위해서 낮은 값으로 설정해보자
    subsample=0.8, # 각 Decision Tree 훈련에 사용할 샘플의 개수를 나타낸다. 80%만 활용해보자.
    max_depth=3, # Decision Tree의 깊이를 나타낸다. 많이 깊어지지 않도록 해보자.
    random_state=42,
    verbose=0
).fit(
    X_train, y_train
)

In [ ]:
from sklearn.metrics import accuracy_score


print('# Training performance')
print('- Gradient Boosting: ', accuracy_score(y_train, model_cat.predict(X_train)))

print('# Validation performance')
print('- Gradient Boosting: ', accuracy_score(y_test, model_cat.predict(X_test)))

# Training performance
- Gradient Boosting:  1.0
# Validation performance
- Gradient Boosting:  0.9605263157894737


반복되는 이야기지만, CatBoost 또한 아주 많은 초매개변수가 존재한다. 보다 자세한 건 [공식 문서](https://catboost.ai/en/docs/references/training-parameters/)를 확인하자